# VFT Preprocessing

## Library settings

In [ ]:
import os, sys, random, time, glob
import urllib
import librosa
import librosa.display
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
from PIL import Image
from tqdm import tqdm

In [ ]:
directory = os.getcwd()
print("Current working directory is:", directory)

## MFCC convert trials

In [ ]:
path = "E:/RESEARCH/BRAIN/research_data/VFT/001/VerbalFluencyTest-001-1Stimulus-2.wav"

In [ ]:
sample_rate=16000

x = librosa.load(path,sample_rate)[0]
S = librosa.feature.melspectrogram(x, sr=sample_rate, n_mels=128)
log_S = librosa.power_to_db(S, ref=np.max)
mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=20)

delta2_mfcc = librosa.feature.delta(mfcc, order=2)

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.specshow(delta2_mfcc)
plt.ylabel('MFCC coeffs')
plt.xlabel('Time')
plt.title('MFCC')
plt.colorbar()
plt.tight_layout()

---

## Dataset settings

In [ ]:
vft_dir = "E:/RESEARCH/BRAIN/research_data/VFT/"
vft_file_names = os.listdir(vft_dir)

In [ ]:
## removing unnecessary files except for the wave file
def removing_unnec_files(path):
    i = 1
    for i in range(0, len(os.listdir(path))):
        [os.remove(file) for file in glob.glob(path + str(i) + '/*.xml')]
        [os.remove(file) for file in glob.glob(path + str(i) + '/*.edat3')]
        [os.remove(file) for file in glob.glob(path + str(i) + '/*.txt')]
        i += 1

In [ ]:
removing_unnec_files(vft_dir)